<a href="https://colab.research.google.com/github/alexandertaoadams/AlexanderAdamsMastersThesis/blob/main/NB_EEG_Epilepsy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Time Series Classification: Epilepsy2**

https://www.timeseriesclassification.com/description.php?Dataset=Epilepsy2

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install gpjax
!pip install sktime

In [ ]:
# jax libraries
import numpy as np
import jax
import jax.numpy as jnp

# gpjax libraries
import gpjax as gpx

# core libraries
from flax import nnx
import optax as ox

# data manipulation and visualisation libraries
import pandas as pd
from sktime.datasets import load_from_tsfile
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
!git clone https://github.com/alexandertaoadams/AlexanderAdamsMastersThesis.git

import sys
sys.path.insert(0, '/content/AlexanderAdamsMastersThesis')
import AlexanderAdamsMastersThesis.src as src

from src.kernels import SignatureKernel
from src.inducing_variables import initial_inducing_variables
from src.utils import display_parameters, display_results, normalise

### **Data Loading and Preprocessing**

In [6]:
file_path_train = "/content/drive/MyDrive/DATA_EEG_Epilepsy/Epilepsy2_TRAIN.ts"
file_path_test = "/content/drive/MyDrive/DATA_EEG_Epilepsy/Epilepsy2_TEST.ts"

train_data, train_labels = load_from_tsfile(file_path_train)
test_data, test_labels = load_from_tsfile(file_path_test)

In [7]:
train_data_2 = jnp.array((np.stack([np.stack(row) for row in train_data.to_numpy()])))
train_labels_2 = jnp.array([int(i) for i in train_labels])
xtrain, train_mean, train_std = normalise(train_data_2)
ytrain = train_labels_2

print(xtrain.shape)
print(ytrain.shape)
print(ytrain)

(80, 1, 178)
(80,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1
 1 1 1 1 1 1]


In [8]:
test_data_2 = jnp.array((np.stack([np.stack(row) for row in test_data.to_numpy()])))
test_labels_2 = jnp.array([int(i) for i in test_labels])
xtest = ((test_data_2 - train_mean) / (train_std))
ytest = test_labels_2

print(xtest.shape)
print(ytest.shape)
print(ytest)

(11420, 1, 178)
(11420,)
[0 0 0 ... 1 1 1]


### **Model Training**



In [9]:
# Initialising model
q_kernel = SignatureKernel(1, 178, 4)
q_mean_function = gpx.mean_functions.Constant()
q_prior = gpx.gps.Prior(mean_function=q_mean_function, kernel=q_kernel)
q_likelihood = gpx.likelihoods.Bernoulli(xtrain.shape[0])
q_posterior = q_likelihood * q_prior

# Create dataset, initialise inducing variables
D = gpx.dataset.Dataset(jnp.reshape(xtrain, (xtrain.shape[0], -1)), jnp.expand_dims(ytrain, axis=1))
Z = initial_inducing_variables(xtrain, ytrain, 8)

# Model
q = gpx.variational_families.VariationalGaussian(
    posterior=q_posterior,
    inducing_inputs=Z
)

/usr/local/lib/python3.12/dist-packages/gpjax/dataset.py:43: UserWarning: X is not of type float64. Got X.dtype=float32. This may lead to numerical instability. 
  _check_precision(self.X, self.y)
/usr/local/lib/python3.12/dist-packages/gpjax/dataset.py:43: UserWarning: y is not of type float64.Got y.dtype=int32. This may lead to numerical instability.
  _check_precision(self.X, self.y)


In [ ]:
# Training
optimised_model, history = gpx.fit(
    model=q,
    objective= lambda model, data: -gpx.objectives.elbo(model, data),
    train_data=D,
    optim=ox.adam(learning_rate=1e-3),
    trainable=gpx.parameters.Parameter,
    num_iters=1000,
    batch_size=80,
    verbose=True
)

### **Model Evaluation**

In [11]:
@jax.jit
def predict_batch(model, batch):
    def predict_single(x):
        latent_dist = model.predict(x[None, :])
        predicted_dist = model.posterior.likelihood(latent_dist)
        return predicted_dist.mean.squeeze()
    return jax.vmap(predict_single)(batch)

def batched_predict(xtest, model, batch_size=90):
    num_points = xtest.shape[0]
    num_batches = (num_points + batch_size - 1) // batch_size

    results = []
    for i in range(num_batches):
        batch = xtest[i * batch_size : (i + 1) * batch_size]
        preds = predict_batch(model, batch)
        results.append(preds)

    return jnp.concatenate(results, axis=0)

In [12]:
# Get predicted means
predicted_mean = batched_predict(xtest, batch_size=100, model=optimised_model)
# Get predicted class labels (0 or 1)
predicted_labels = jnp.round(predicted_mean)

# print("Predicted means:", predicted_mean)
# print("Predicted labels:", predicted_labels)
# print("True labels:", ytest)

In [13]:
# Display results
results = display_results(predicted_labels, ytest)
results

,Value
Metric,
Test Size,11420.000
Negative Samples,2260.000
Postive Samples,9160.000
MCC,0.567
F1 score,0.924
Precision,0.894
Recall,0.956
